<a href="https://colab.research.google.com/github/dangdang2222/LG_Aimers/blob/main/Y_5%2C6%2C7%2C13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import random
import os
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,13): # ignore 'ID'
        rmse = mean_squared_error(np.array(gt)[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(np.array(gt)[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score
def lg_nrmse_one(gt, preds):
    # Y_08의 경우는 가중치 1.2 부여. 8 이후부터는 모두 가중치 1.0
    all_nrmse = []
    rmse = mean_squared_error(np.array(gt), preds, squared=False)
    nrmse = rmse/np.mean(np.abs(np.array(gt)))
    all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse)
    return score

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/LG_Aimer/train.csv').drop(columns="ID")
train_x = train_df.filter(regex='X') # Input : X Featrue
train_x=train_x.drop(columns=['X_04','X_23','X_47','X_48'])
train_y = train_df.filter(regex='Y') # Output : Y Feature

In [14]:
train_y06 = train_y['Y_06']
train_y06
train_y06=pd.DataFrame(train_y06, index= train_y.index)

In [6]:
from sklearn.model_selection import train_test_split

# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x.quantile(0.01)
p99 = train_x.quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x = train_x.clip(p01, p99, axis=1)

In [7]:
train_x['X__21'] = (train_x['X_24'] - train_x['X_12'])
train_x['X__22'] = (train_x['X_25'] - train_x['X_12'])
train_x['X__23'] = (train_x['X_26'] - train_x['X_12'])
train_x['X__24'] = (train_x['X_27'] - train_x['X_12'])
train_x['X__25'] = (train_x['X_28'] - train_x['X_12'])
train_x['X__26'] = (train_x['X_29'] - train_x['X_12'])

In [8]:
import xgboost as xgb
regressor1= xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

## Y_06

In [9]:
    x_train,x_valid, y_train,y_valid = train_test_split(train_x,train_y06,test_size=0.3) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)


[10:29:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.12698733705760756


In [15]:
new_col = ['X_49', 'X_09', 'X_14', 'X_05', 'X_15', 'X_07', 'X_03', 'X_02', 'X_38', 'X_16', 'X_01', 'X_50', 'X_56', 'X_08', 'X_44', 'X_22', 'X_41', 'X_21','X_42']
train_x_xgb = train_x[new_col]



In [16]:
train_x_xgb.columns

Index(['X_49', 'X_09', 'X_14', 'X_05', 'X_15', 'X_07', 'X_03', 'X_02', 'X_38',
       'X_16', 'X_01', 'X_50', 'X_56', 'X_08', 'X_44', 'X_22', 'X_41', 'X_21',
       'X_42'],
      dtype='object')

In [19]:
    x_train,x_valid, y_train,y_valid = train_test_split(train_x_xgb,train_y06,test_size=0.3,random_state=42) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)

[12:08:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.12298070045135383


## Y_07

In [20]:
train_y07 = train_y['Y_07']
train_y07
train_y07=pd.DataFrame(train_y07, index= train_y.index)

In [16]:
    x_train,x_valid, y_train,y_valid = train_test_split(train_x,train_y07,test_size=0.3) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)


[11:03:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.15616743082976933


In [21]:
new_col = ['X_13', 'X_19', 'X_32', 'X_07', 'X_09', 'X_49', 'X_21', 'X_03', 'X_17', 'X__22', 'X_05', 'X_22', 'X__26', 'X_08', 'X_51', 'X__23', 'X_31', 'X_14', 'X_44', 'X__25', 'X_38', 'X__21', 'X_52','X_40']
train_x_xgb = train_x[new_col]


In [22]:
    x_train,x_valid, y_train,y_valid = train_test_split(train_x_xgb,train_y07,test_size=0.3,random_state=42) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)

[12:08:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.15573291235781145


## Y_05

In [5]:
train_y05 = train_y['Y_05']
train_y05
train_y05=pd.DataFrame(train_y05, index= train_y.index)

In [9]:
    x_train,x_valid, y_train,y_valid = train_test_split(train_x,train_y05,test_size=0.3,random_state=42) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)


[11:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.09563775884496022


In [13]:
    new_col = ['X_49', 'X_56', 'X_55', 'X_54', 'X_53', 'X__26', 'X__25', 'X__24', 'X__23', 'X__22', 'X_52', 'X_51', 'X_50', 'X__21', 'X_46', 'X_40', 'X_45', 'X_44', 'X_43', 'X_42', 'X_41', 'X_39', 'X_38', 'X_37', 'X_36', 'X_35', 'X_34', 'X_33', 'X_32', 'X_31', 'X_30', 'X_29', 'X_28', 'X_22', 'X_27', 'X_26', 'X_25', 'X_24', 'X_21', 'X_20', 'X_19', 'X_18', 'X_17', 'X_16', 'X_15', 'X_14', 'X_13', 'X_09', 'X_12', 'X_08', 'X_07', 'X_06', 'X_05', 'X_03', 'X_02','X_01']
    train_x_xgb = train_x[new_col]

    x_train,x_valid, y_train,y_valid = train_test_split(train_x_xgb,train_y05,test_size=0.3,random_state=42) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)

[12:05:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.09558576195193111


## Y_13

In [23]:
train_y13 = train_y['Y_13']
train_y13
train_y13=pd.DataFrame(train_y13, index= train_y.index)

In [24]:
    x_train,x_valid, y_train,y_valid = train_test_split(train_x,train_y13,test_size=0.3,random_state=42) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)

[12:09:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.028819865420023485


In [25]:
    new_col = ['X_20', 'X_32', 'X_30', 'X_07', 'X_49', 'X_03', 'X_21', 'X_22', 'X_09', 'X_08', 'X_19', 'X__23', 'X_05', 'X_45', 'X_06', 'X_14', 'X_54', 'X_51', 'X_55', 'X_53', 'X_52', 'X__25', 'X_16', 'X__21', 'X_40', 'X_25', 'X_42']
    train_x_xgb = train_x[new_col]

    x_train,x_valid, y_train,y_valid = train_test_split(train_x_xgb,train_y13,test_size=0.3,random_state=42) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    print(score)

[12:23:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.02873849641625251
